<a href="https://colab.research.google.com/github/sjmin99/-/blob/master/%EB%8B%B9%EB%87%A8%EB%B3%91kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import torch.optim as optim 

In [2]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

!kaggle competitions download -c logistic-classification-diabetes
!unzip logistic-classification-diabetes

-rw-r--r-- 1 root root 63 May  3 11:43 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 9.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=9b179527e2eef106d1fdc864f69503b97a3114189857fe7f22d80984ebc0bf6f
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6
  0% 0.00/16.3k [00:00<?, ?B/s]
100% 16.3k/16.3k [00:00<00:00, 14.3MB/s]
Archive:  logistic-classification-diabetes.zip
  inflating: submission_form.csv     
  inflating: test_data.csv           
  inflating: train.csv               


In [0]:
torch.manual_seed(1)

pd_data = pd.read_csv('train.csv')
x_train = torch.FloatTensor(np.array(pd_data.iloc[:,1:9]))
y_train = torch.LongTensor(np.array(pd_data.iloc[:,9]))

In [115]:
W = torch.zeros((8,2), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

epochs = 10000
lr = 1e-3

optimizer = optim.SGD((W,b), lr = lr, momentum=0.9, nesterov= True)

for epoch in range(epochs+1):

  H = F.softmax(x_train.matmul(W)+b,dim=1)
  y_onehot = torch.zeros(len(y_train),2)
  y_onehot = y_onehot.scatter_(1,y_train.unsqueeze(1),1)


  cost = (y_onehot * -torch.log(H)).sum(dim=1).mean()


  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 1000 == 0:
    print("Epoch : {:4d}/{}, Cost : {:.6f}".format(
        epoch, epochs, cost.item()))
    

Epoch :    0/10000, Cost : 0.693147
Epoch : 1000/10000, Cost : 0.539839
Epoch : 2000/10000, Cost : 0.507774
Epoch : 3000/10000, Cost : 0.493156
Epoch : 4000/10000, Cost : 0.485530
Epoch : 5000/10000, Cost : 0.481176
Epoch : 6000/10000, Cost : 0.478525
Epoch : 7000/10000, Cost : 0.476828
Epoch : 8000/10000, Cost : 0.475695
Epoch : 9000/10000, Cost : 0.474911
Epoch : 10000/10000, Cost : 0.474351


In [0]:
pd_test = pd.read_csv('test_data.csv')

In [0]:
x_test = torch.FloatTensor(np.array(pd_test.iloc[:,1:9]))
H = x_test.matmul(W)+b
# predict = torch.argmax(H,dim=1)

Id = np.array([i for i in range(len(x_test))]).reshape(-1,1)

Expected = predict.detach().numpy().reshape(-1,1)
result = np.hstack([Id,Expected])

df = pd.DataFrame(result, columns=["Id","Label"])
df = df.astype(int)
# df.to_csv("result.csv", index=False, header= True)
df.to_csv("result.csv", index=False, header= True)

In [118]:
!kaggle competitions submit -c logistic-classification-diabetes -f result.csv -m "Assignment"

100% 249/249 [00:03<00:00, 69.3B/s]
Successfully submitted to logistic classification : diabetes